动态在pathlib中插入src

In [1]:
import sys
import os
from IPython.display import display
from wcmatch import pathlib

display(sys.executable)
src_p = pathlib.Path("..") / "src"
if src_p.exists() and src_p.is_dir():
    print(f'insert {src_p.resolve()} in PythonPath')
    sys.path.insert(0, os.fspath(src_p.resolve()))

'/Users/lhc/.pyenv/versions/data/bin/python3.8'

insert /Users/lhc/dev/repo/prac_py/src in PythonPath


# 1 context manager

# 2 Function

In [1]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [2]:
def reverse(word):
    return word[::-1]
reverse('hello')
sorted(fruits,key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## 2.1 map()
特别是一个序列做map,直接用生成器表达式替换.

In [8]:
def factorial(n):
    return 1 if n<2 else n*factorial(n-1)

display(list(map(factorial, range(6))))
[factorial(n) for n in range(6) if n%2]

[1, 1, 2, 6, 24, 120]

[1, 6, 120]

## 2.2 函数内置属性

In [19]:
for n in dir(factorial):
    print(n,end='/')
print('\n')
fac = factorial
print(fac.__hash__())

__annotations__/__call__/__class__/__closure__/__code__/__defaults__/__delattr__/__dict__/__dir__/__doc__/__eq__/__format__/__ge__/__get__/__getattribute__/__globals__/__gt__/__hash__/__init__/__init_subclass__/__kwdefaults__/__le__/__lt__/__module__/__name__/__ne__/__new__/__qualname__/__reduce__/__reduce_ex__/__repr__/__setattr__/__sizeof__/__str__/__subclasshook__/

283942422


## 2.3 函数的参数
flutpy C5.7的拆解
### 仅限关键字键值对的函数

In [13]:
def f(a,*,b=1):
    return a,b
f(1,b=9)

(1, 9)

In [14]:
fac = factorial
display(f.__code__) # 属性非方法
display(f.__defaults__) # 为什么是None,仅限关键字参数?
display(fac.__defaults__)
f.__kwdefaults__

<code object f at 0x10750d3a0, file "<ipython-input-13-bd6c10d9a2f7>", line 1>

None

None

{'b': 1}

In [11]:
from inspect import signature
sig = signature(f)
display(str(sig))
for name,p in sig.parameters.items():
     print(f'{name}, default value:{p.default}, kind:{p.kind}')

'(a, *, b=1)'

a, default value:<class 'inspect._empty'>, kind:1
b, default value:1, kind:3


In [22]:
r = f(a=0,b=8)
r[0]

0

### star assignment,Py 3.5+
PEP 448,Additional Unpacking Generalizations / PEP 3132 -- Extended iterable unpacking

In [28]:
a=1,2
print(type(a),a)
*a,=[[1,2,3],[4,5,6]]
print(type(a),a)

<class 'tuple'> (1, 2)
<class 'list'> [[1, 2, 3], [4, 5, 6]]


In [36]:
# Whilst *elements, = iterable causes elements to be a list, elements = *iterable, causes elements to be a tuple.
# *b, 类似于函数定义 fun(*arg,**kwargs)), * in left side means catch everything that isn't assigned to a name and assign it to the starred expression.
*b,=3,4,5
print(type(b),b) 
for i in b:
    print(type(i),end='')
print('\n')
*b,=(6,7,8) #拿出tuple元素往*b中填,形成list
print(type(b),b)
for i in b:
    print(type(i),end='')
print('\n')
*b,c,d=3,4,5
print(type(b),b)

iterable = [1,2,3,4]
b = *iterable, #可看做 b=1,2,3,4,It expands the contents of the iterable it is attached to.
print(type(b),b)
b=[*iterable]
print(type(b),b)

<class 'list'> [3, 4, 5]
<class 'int'><class 'int'><class 'int'>

<class 'list'> [6, 7, 8]
<class 'int'><class 'int'><class 'int'>

<class 'list'> [3]
<class 'tuple'> (1, 2, 3, 4)
<class 'list'> [1, 2, 3, 4]


### *可变参数

In [74]:
def fun(a,*num):
    print(type(num),num)
fun(1) # 变长参数会转换成turple,这和*iterable是一直的,都是turple
fun(1,2,3,4) 
fun([1,2,3,4])

iterable = [1,2,3,4]
fun(*iterable)
iterable=([1,2],[3,4],5)
fun(*iterable)
iterable = {1:1,2:2,3:3,4:4} #*只解包dict的key
fun(*iterable)

<class 'tuple'> ()
<class 'tuple'> (2, 3, 4)
<class 'tuple'> ()
<class 'tuple'> (2, 3, 4)
<class 'tuple'> ([3, 4], 5)
<class 'tuple'> (2, 3, 4)


### ***关键字参数

In [40]:
def fun(a,**kw):
    print(kw)
fun(1,name='lhc',age='30')
iterable = {'one':1,'two':2,'three':3,'four':4} 
fun(1,**iterable)
iterable = {1:1,2:2,3:3,4:4} #fun() keywords must be strings,即变量名必须是strings
fun(1,**iterable)

{'name': 'lhc', 'age': '30'}
{'one': 1, 'two': 2, 'three': 3, 'four': 4}


TypeError: fun() keywords must be strings

### 可变参数*之前的参数不能指定参数名

In [3]:
def fun(a=0,*num):
    print(a,num)
fun(1,2,3)
fun(1,2,3,num=9) # TypeError: fun() got an unexpected keyword argument 'num'

1 (2, 3)


TypeError: fun() got an unexpected keyword argument 'num'

In [1]:
fun(a=1,2,3) #SyntaxError: positional argument follows keyword argument

SyntaxError: positional argument follows keyword argument (<ipython-input-1-2965dcb9a2b8>, line 1)

### 可变参数*之后的参数必须指定参数名

In [57]:
def fun(a,*b,c=None):
    print(a,b,c)
fun(1,2,3,4) # 若c没有默认值c=None:TypeError: fun() missing 1 required keyword-only argument: 'c'
fun(1,2,3,c=1)

1 (2, 3, 4) None
1 (2, 3) 1


### 关键字参数只能作为最后一个参数
关键字前面的参数按照位置赋值还是名称赋值都可以.  
**顺序：位置参数（必选参数），默认参数，单星号参数或星号分隔符，关键字参数，双星号参数；**  
默认参数和关键字参数很像,不要搞混,默认参数在加载时即赋值.

In [79]:
def fun(a,*b,c=None,**d):
    print(a,b,c,d)
fun(1,2,3,4,m=5,n=6)
fun(1)
fun(1,2)
fun(1,2,q=0)

1 (2, 3, 4) None {'m': 5, 'n': 6}
1 () None {}
1 (2,) None {}
1 (2,) None {'q': 0}


### 解包作为参数传入函数

In [70]:
def fun(a,b):
    print(a,b)
iterable = (1,2)
fun(*iterable)
iterable = [1,2,3,4]
fun(*iterable) # TypeError: fun() takes 2 positional arguments but 4 were given

1 2


TypeError: fun() takes 2 positional arguments but 4 were given

In [69]:
person1 = {'name':'lhc','age':30}
print("{name}'s age is {age}".format(**person1))

lhc's age is 30


## 2.4 函数体也有属性

In [7]:
def addIt(x,y):
    try:
        addIt.counter +=1
    except AttributeError:
        addIt.counter = 0
    print(f'execute {addIt.counter}')
    return x+y
addIt.a = 0
f = addIt
display(f(1,2))
print(f.a,f.counter)
addIt.a+=2
f = addIt
display(f(3,4))
print(f.a,f.counter)
addIt.__dict__

execute 0


3

0 0
execute 1


7

2 1


{'a': 2, 'counter': 1}

## 2.5支持函数式编程的包
高阶函数演示.

In [50]:
from functools import reduce
from operator import mul, xor

def fact_re(n):
    """ reduce 结合 mul 计算阶乘 """
    return reduce(mul,range(1,n+1))

fact_re(5)

120

### itemgetter()

In [2]:
from operator import itemgetter
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
cc_name = itemgetter(1,0)
for c in metro_data:
     print(cc_name(c))

itemgetter(3)(metro_data[3])

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


(40.808611, -74.020386)

### attrgetter() and namedtuple()

In [4]:
from operator import attrgetter
from collections import namedtuple

# 参数1是namedtuple的值
LatLong = namedtuple('LL','lat long')
Metro = namedtuple('Metro','name cc pop corrd')
# name,cc,pop,(lat,long) 对元组进行拆包,并赋予名称
# 之后给到namedtuple:Metro
metro_area = [Metro(name,cc,pop,LatLong(lat,long)) for name,cc,pop,(lat,long) in metro_data]

attrgetter('name','corrd.lat','corrd.long')(metro_area[2])

('Mexico City', 19.433333, -99.133333)

### functools.partial

In [7]:
from operator import mul
from functools import partial

triple = partial(mul,3)
display(triple(7))
display(list(map(triple,range(1,11))))

21

[3, 6, 9, 12, 15, 18, 21, 24, 27, 30]

### function 实现 策略模式

In [14]:
from fluentpy.c6.order import Customer
from fluentpy.c6.order import LineItem
from fluentpy.c6.order import Order
from fluentpy.c6.order import fidelity_promo

lhc = Customer('LHC48052', 999)
cart = [LineItem('banana',4,0.5),LineItem('apple',3,0.4)]
display(Order(lhc,cart,fidelity_promo))

cart = [LineItem('orange',5,0.7)]
display(Order(lhc,cart,fidelity_promo))

cart = 'abc'
for i in cart:
    print(type(i))
    print(i)
cart = list(list(cart))
for i in cart:
    print(type(i))
    print(i)

fidelity:999


<Order total:3.20 due: 3.20>

fidelity:999


<Order total:3.50 due: 3.50>

<class 'str'>
a
<class 'str'>
b
<class 'str'>
c
<class 'str'>
a
<class 'str'>
b
<class 'str'>
c


# 3 Iterator and Generator
From Fluent Python Chapter 14.

## Iterable 和 Iterator

In [4]:
from fluentpy.c14.sentence import Sentence1

s = Sentence1("Crazy world, extraordinary world.")
display(s)
i = iter(s)
for word in i:
    print(word)

for word in s:
    print(word)

# i 中已无可迭代的内容.
next(i)

Sentence 'Crazy world,...dinary world.'

Crazy
world
extraordinary
world
Crazy
world
extraordinary
world


StopIteration: 

In [20]:
from fluentpy.c14.sentence import Sentence2

s = Sentence2("Crazy world, extraordinary world.")
for word in s:
    print(word)

# iter()每次构造新的迭代器
display(next(iter(s)))
display(next(iter(s)))

# 新的迭代器,重新开始
# while 不会捕捉stopiteration异常
i = iter(s)
while(True):
    display(next(i))



Crazy
world
extraordinary
world


'Crazy'

'Crazy'

'Crazy'

'world'

'extraordinary'

'world'

StopIteration: 

## 迭代器中使用生成器函数 yield

In [7]:
from fluentpy.c14.sentence import Sentence3

s = Sentence3("Sentence3, Crazy world, extraordinary world.")
for word in s:
    print(word)

next(iter(s))

in Sentence 3.
Sentence3
in Sentence 3.
Crazy
in Sentence 3.
world
in Sentence 3.
extraordinary
in Sentence 3.
world
in Sentence 3.


'Sentence3'

In [6]:
from fluentpy.c14.sentence import Sentence4

s = Sentence4("Sentence 4, Crazy world, extraordinary world.")
for word in s:
    print(word)

Sentence
4
Crazy
world
extraordinary
world
end
with
sentence
4


'Sentence'

## 生成器函数:yield from(广度和深度搜索)

In [5]:
from pycookbook.c4.node44 import Node

root = Node(0)
c1 = Node(1)
c2 = Node(2)
c3 = Node(3)

root.add_child(c1)
root.add_child(c2)
root.add_child(c3)

c1.add_child(Node(11))
c1.add_child(Node(12))
c2.add_child(Node(21))
c3.add_child(Node(31))

for ch in root.depth_first():
    print(f'out print:{ch}')

print('======================')
for ch in root.depth_first_yield_from():
    print(f'out print:{ch}')

print('======================')
isRoot = True
for ch in root.width_first(isRoot):
    print(f'out print:{ch}')
    isRoot = False

before yield:Node(0)
out print:Node(0)
after yield:Node(0)
in for Node(1)
before yield:Node(1)
Node(1)
after yield:Node(1)
in for Node(11)
before yield:Node(11)
Node(11)
after yield:Node(11)
in for Node(12)
before yield:Node(12)
Node(12)
after yield:Node(12)
in for Node(2)
before yield:Node(2)
Node(2)
after yield:Node(2)
in for Node(21)
before yield:Node(21)
Node(21)
after yield:Node(21)
in for Node(3)
before yield:Node(3)
Node(3)
after yield:Node(3)
in for Node(31)
before yield:Node(31)
Node(31)
after yield:Node(31)
inner print:before yield:Node(0)
out print:Node(0)
inner print:after yield:Node(0)
inner print:before yield:Node(1)
out print:Node(1)
inner print:after yield:Node(1)
inner print:before yield:Node(11)
out print:Node(11)
inner print:after yield:Node(11)
inner print:before yield:Node(12)
out print:Node(12)
inner print:after yield:Node(12)
inner print:before yield:Node(2)
out print:Node(2)
inner print:after yield:Node(2)
inner print:before yield:Node(21)
out print:Node(21)
inn

# 4. Decroator

## 4.1 装饰器加载即运行
函数的默认参数也是这个在装载时运行.

In [2]:
import fluentpy.c7.regi as re



run register(<function f1 at 0x10d7f0e50>)
run register(<function f2 at 0x10d877940>)


In [4]:
# 导入后registry即有值
display(re.registry)
# 装饰器返回的还是原函数,所以f1的元数据保持不变.
display(re.f1.__name__)
re.f1()

[<function fluentpy.c7.regi.f1()>, <function fluentpy.c7.regi.f2()>]

'f1'

running f1()


In [6]:
import fluentpy.c7.average as av

# make_sum_average()运行后,生产出average
mav = av.make_sum_average()
display(mav(10))
display(mav(15))
mav.__code__.co_varnames

10.0

12.5

('new_value', 'total')

In [4]:
mav.__code__.co_freevars

('series',)

In [5]:
# __closure__中保存着闭包的元素,本例中是series,cell_contents是series的值.
mav.__closure__[0].cell_contents

[10, 15]

In [16]:
mav = av.make_average()
mav(10)
mav(12)

11.0

In [10]:
def x1():
    count += 1
    return count
x1()

UnboundLocalError: local variable 'count' referenced before assignment

## 简单装饰器 timeit

In [3]:
import fluentpy.c7.usetimeit as ut

print('*'*20,'Calling snooze(0.5)')
ut.snooze(0.5)
print('*'*20,'Calling factorial(6),6!=')
ut.factorial(6)
# 装饰器没有使用@wraps(),函数原来的信息丢失.
print(f'After decroator,factorial --> {ut.factorial.__name__}')

******************** Calling snooze(0.5)
00.50406557 snooze(0.5) --> None
******************** Calling factorial(6),6!=
00.00000097 factorial(1) --> 1
00.00007083 factorial(2) --> 2
00.00012478 factorial(3) --> 6
00.00017559 factorial(4) --> 24
00.00022486 factorial(5) --> 120
00.00027752 factorial(6) --> 720
After decroator,factorial --> clocked


In [11]:
import fluentpy.c7.usetimeit as ut

ut.fibonacci(5)

00.00001494 fibonacci(1) --> 1
00.00000086 fibonacci(0) --> 0
00.00000080 fibonacci(1) --> 1
00.00008431 fibonacci(2) --> 1
00.00043836 fibonacci(3) --> 2
00.00000041 fibonacci(0) --> 0
00.00000043 fibonacci(1) --> 1
00.00323095 fibonacci(2) --> 1
00.00000070 fibonacci(1) --> 1
00.00000075 fibonacci(0) --> 0
00.00000091 fibonacci(1) --> 1
00.00006189 fibonacci(2) --> 1
00.00013084 fibonacci(3) --> 2
00.00362917 fibonacci(4) --> 3
00.00411098 fibonacci(5) --> 5


5

In [9]:
import fluentpy.c7.usetimeit as ut

ut.fibonacci_cache(10)

00.00000214 fibonacci_cache(10) --> 55


55

## 带参数的装饰器1
fluentp C7-22.  
注意闭包的自由变量registry作用域.

In [2]:
import fluentpy.c7.regi_param as regi_p

# 加载即运行,但装饰器每次一个新的register,导致registry没有累加
# 如果需要不断累加,像fluentpy C7.22中一样,把registry定义在module级别
# 这个主要想看下如何获取闭包的变量,如pycookbook C7.12

registry:4504258592 it's len->0
running register(active=False) -> decorate(<function f1 at 0x10c79cc10>)
complete regi, registry:4504258592 it's len->0
r:4504257696 it's len->0
registry:4504258592 it's len->0
running register(active=False) -> decorate(<function f2 at 0x10c7ad160>)
complete regi, registry:4504258592 it's len->0
r:4504257696 it's len->0
registry:4504258592 it's len->0
running register(active=True) -> decorate(<function f4 at 0x10c7ad700>)
complete regi, registry:4504258592 it's len->1
r:4504257696 it's len->1
registry:4504258592 it's len->1
running register(active=True) -> decorate(<function f5 at 0x10c7adc10>)
complete regi, registry:4504258592 it's len->2
r:4504257696 it's len->2


In [3]:
f = regi_p.register
display(f)
# 手动注册,一直使用装饰器的引用
deco = f(active=True)
display(deco)
f_run = deco(regi_p.f1)
display(f_run)
# run f1()
f_run()
display(len(deco.get_registry()))
f_run = deco(regi_p.f3)
display(len(deco.get_registry()))
# run f3()
f_run()

<function fluentpy.c7.regi_param.register(active=False, r={<function f4 at 0x10c7ad700>, <function f5 at 0x10c7adc10>})>

registry:4504258592 it's len->2


<function fluentpy.c7.regi_param.register.<locals>.decorate(func)>

running register(active=True) -> decorate(<function f1 at 0x10c79cc10>)
complete regi, registry:4504258592 it's len->3
r:4504257696 it's len->3


<function fluentpy.c7.regi_param.f1()>

running f1()


3

running register(active=True) -> decorate(<function f3 at 0x10c7ad040>)
complete regi, registry:4504258592 it's len->4
r:4504257696 it's len->4


4

running f3()


In [4]:
g = regi_p.register
deco = g(active=True)
f_run= deco(regi_p.f3)
f_run()

registry:4504258592 it's len->4
running register(active=True) -> decorate(<function f3 at 0x10c7ad040>)
complete regi, registry:4504258592 it's len->4
r:4504257696 it's len->4
running f3()


## 带参数的装饰器2
pycookbook C9.2和C9.4

In [8]:
from pycookbook.c9.logged import *
import logging

logging.basicConfig(level=logging.DEBUG)
x = add
display(x)
display(x.__wrapped__)
display(x(2,3))
x(4,5)

<function pycookbook.c9.logged.add(x, y)>

<function pycookbook.c9.logged.add(x, y)>

5

9

解构logged这个装饰器的运行机制

In [7]:
from pycookbook.c9.logged import *
import logging

logging.basicConfig(level=logging.DEBUG)

def addIt(x,y):
    return x+y

logged(logging.CRITICAL)(addIt)(2,3)

f = logged(logging.CRITICAL)
display(f)
f = f(addIt)
# 由于@wraps的存在,这里保留了addIt的元信息.
# 否则返回: <function pycookbook.c9.logged.logged.<locals>.decorate.<locals>.wrapper(*args, **kwargs)>
# 如果返回了被decorate的func,就可以用实际参数进行调用了.
display(f)
f(3,4)

CRITICAL:__main__:addIt
wraps(func)->functools.partial(<function update_wrapper at 0x103b0c310>, wrapped=<function addIt at 0x1071f98b0>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
wrapper-><function addIt at 0x1071f9700>
wraps(func)(wrapper)-><function addIt at 0x1071f9700>


<function pycookbook.c9.logged.logged.<locals>.decorate(func)>

wraps(func)->functools.partial(<function update_wrapper at 0x103b0c310>, wrapped=<function addIt at 0x1071f98b0>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
wrapper-><function addIt at 0x1071f9310>
wraps(func)(wrapper)-><function addIt at 0x1071f9310>


<function __main__.addIt(x, y)>

CRITICAL:__main__:addIt


7

## 带参数的装饰器3
参数可选:pycb C9.6

In [2]:
from pycookbook.c9.loggeds import *
import logging

logging.basicConfig(level=logging.WARNING)

@loggeds
def addIt(x,y):
    return x+y



Condition func not None, execute loggeds with func...
attach_method is None. get attatch_wrapper...
attach_method is None. get attatch_wrapper...


In [3]:
display(addIt(2,3))
# log在函数定义时先执行了,等待addParam(3,3)的时候再执行.
print('*'*10)

@loggeds(level=logging.CRITICAL,message='addParam test',name='param example')
def addParam(x,y):
    return x+y

# 以下访问闭包属性
f = addParam
display(f.logmsg)
display(f(3,3))
f.set_msg('new_msg')
display('静态用法:',f.logmsg)
display('用函数:',f.msg_it())
display('动态用法:',f.get_msg())
display(f(4,4))

5

**********
Condition func=None, return func loggeds...
Condition func not None, execute loggeds with func...
attach_method is None. get attatch_wrapper...
attach_method is None. get attatch_wrapper...


'addParam test'

CRITICAL:param example:addParam test


6

...

'new_msg'

'动态用法:'

'new_msg'

CRITICAL:param example:new_msg


8

In [6]:
from pycookbook.c9.loggeds import *
import logging

loggeds运行时机和装饰器机制有关.见装饰器2的结构部分  
相当于loggeds(addIt),运行loggeds一次,func is not None

In [8]:
@loggeds
def addIt(x,y):
    return x+y

Condition func not None,return func loggeds...
get attatch_wrapper...
get attatch_wrapper...


In [9]:
addIt(3,4)

DEBUG:__main__:addIt


7

相当于loggeds(level=...)(addParam)  
第一次运行func is None,返回partial(loggeds,...)即还是loggeds,再次运行时func is not None

In [7]:

@loggeds(level=logging.CRITICAL,message='addParam test',name='param example')
def addParam(x,y):
    return x+y

Condition func=None,return func loggeds...
Condition func not None,return func loggeds...
get attatch_wrapper...
get attatch_wrapper...


## 将装饰器定义为类pycb C9.9

### 类装饰器作用在方法上,起调__call__()

In [3]:
from pycookbook.c9.profiled import Profiled

@Profiled
def add(x,y):
    return x+y

in __init__(),wraps(func),f:functools.partial(<function update_wrapper at 0x104cc6310>, wrapped=<function add at 0x1081b6c10>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
**********
self: <pycookbook.c9.profiled.Profiled object at 0x1082cd340>
**********
in __init__(),f(self),f:<pycookbook.c9.profiled.Profiled object at 0x1082cd340>


In [4]:
add(3,4)

in __call__()...
ncalls:... 1


7

### 将类装饰器装饰在类的方法上

In [3]:
from pycookbook.c9.profiled import Profiled

def addIt(x,y):
    return x+y

p = Profiled(addIt)
print(f'***p:{p},p.__wrapped__:{p.__wrapped__}')
# p有__call__()函数,可调用,addIt(3,4)等同于p(3,4)
p(3,4)

===1===in __init__(),wraps(func)->functools.partial(<function update_wrapper at 0x103329310>, wrapped=<function addIt at 0x104f82c10>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
===2===self:<pycookbook.c9.profiled.Profiled object at 0x107128fa0>
===3===in __init__(),f(self)-><pycookbook.c9.profiled.Profiled object at 0x107128fa0>
***p:<pycookbook.c9.profiled.Profiled object at 0x107128fa0>,p.__wrapped__:<function addIt at 0x104f82c10>
===4===in __call__(),ncalls:...1


7

In [9]:
from pycookbook.c9.profiled import Profiled

class Spam:
    def bar(self,x):
        print(self,x)
    p = Profiled(bar)
s = Spam()
#p = Profiled(Spam.bar)
#p.__get__(s,Spam)(3) #相当于在bar前定义@Profiled
s.p(3) #这样写更清晰的表示p是描述器,只不过__get__()把p做为可执行函数绑定给Spam对象,并且p是类变量,只有一个.
s1 = Spam()
s1.p(4) #记录bar在不同对象中被调用的次数.

===1===in __init__(),wraps(func)->functools.partial(<function update_wrapper at 0x10e64c310>, wrapped=<function Spam.bar at 0x113b869d0>, assigned=('__module__', '__name__', '__qualname__', '__doc__', '__annotations__'), updated=('__dict__',))
===2===self:<pycookbook.c9.profiled.Profiled object at 0x1135d9c10>
===3===in __init__(),f(self)-><pycookbook.c9.profiled.Profiled object at 0x1135d9c10>
===5===in __get__(),self-><pycookbook.c9.profiled.Profiled object at 0x1135d9c10>,instance-><__main__.Spam object at 0x1135d9a00>
===4===in __call__(),ncalls:...1
<__main__.Spam object at 0x1135d9a00> 3
===5===in __get__(),self-><pycookbook.c9.profiled.Profiled object at 0x1135d9c10>,instance-><__main__.Spam object at 0x1135d9f40>
===4===in __call__(),ncalls:...2
<__main__.Spam object at 0x1135d9f40> 4


## 将装饰器作用在类方法上pycb C8.10
比pycb C8.10@property 延迟计算简单. C8.10把@property的内容也附加上.

In [ ]:
from functools import wraps
def profileds(fun):
    ncall=0

    @wraps(fun)
    def wrapper(*args,**kwargs):
        nonlocal ncall
        ncall+=1
        return fun(*args,**kwargs)

    wrapper.ncall=lambda:ncall

    return wrapper

class Spam:
    @profileds
    def bar(self,x):
        print(self,x)

s = Spam()
s.bar(3)
s.bar.ncall()

## 使用装饰器动态添加函数的参数 pycb C9.11

In [2]:
from pycookbook.c9.option_debug import *

@optional_debug
def add(x,y):
    return x+y

In [3]:
display(add(2,3))

Calling add


5

In [4]:
import inspect
display(inspect.signature(add))

<Signature (x, y, *, debug=True)>

## 使用装饰器扩展类的功能pycb C9.12
装饰器作用在类之上.

In [ ]:
from pycookbook.c9.c912 import *

@log_getattri
class A:
    def __init__(self,x):
        self.x = x
    def spam(self):
        pass
    def __getattr__(self,name):
        return 'default'

a = A(40)
display(a.x)
display(a.y) #虽然重写了__getattribute__(),会捕捉attributeerror,不存在的属性会自动调用__getattr__()

# 5. Object Oriented

## 函数参数是默认参数时
HauntedBus隐藏的bug.

In [3]:
from fluentpy.c8.HauntedBus import HauntedBus

# 初始化有参数时无任何问题
bus1 = HauntedBus(['lhc'])
display(bus1.passengers)
bus1.pick('gxt')
bus1.drop('lhc')
display(bus1.passengers)

['lhc']

['gxt']

In [5]:
bus2 = HauntedBus()
bus2.pick('haunt')
display(bus2.passengers)

# 函数默认值在定义函数时计算,即在module加载时进行
bus3 = HauntedBus()
display(bus3.passengers)

['haunt', 'haunt']

['haunt', 'haunt']

## SchoolBus 
正确的做法.

In [4]:
from fluentpy.c8.SchoolBus import SchoolBus

bus1 = SchoolBus()
bus1.pick('lhc')
display(bus1.passengers)

bus2 = SchoolBus()
bus2.pick('gcq')
display(bus2.passengers)

['lhc']

['gcq']

## 属性及属性覆盖
每次属性查找，这个协议的方法实际上是由对象的特殊方法__getattribute__()调用。每次通过点号（ins.attribute）或者 
getattr(ins, ‘attribute’)函数调用都会隐式的调用__getattribute__()，它的属性查找的顺序如下：
1. 若判断是描述器优先于实例变量
2. 查看该属性是否能在实例对象的dict中找到
3. 查看该属性是否为实例的类对象的非数据描述符(只实现了__get__(),可以理解成描述器退化成类的属性)
4. __getattri__()方法
关联描述器 Stock1 例子

In [52]:
from fluentpy.c8.simple_bird import *Chicken, Chicken, Chicken, Chicken, Bird, Bird, Chicken, Chicken, Chicken, Bird, Chicken,  

summer = Chicken(2)
print('=====>summer')
print(summer.__dict__)
print(vars(summer)) #vars()返回__dict__属性
print(summer.fly)
print('=====>Chicken')
print(Chicken.__dict__)
print(vars(Chicken))
print('=====>Bird')
print(Bird.__dict__)
print(vars(Bird))

=====>summer
{'age': 2}
{'age': 2}
=====>Chicken
{'__module__': 'fluentpy.c8.simple_bird', 'fly': False, '__init__': <function Chicken.__init__ at 0x1103c69d0>, 'chirp': <function Chicken.chirp at 0x1103c6a60>, '__doc__': None}
{'__module__': 'fluentpy.c8.simple_bird', 'fly': False, '__init__': <function Chicken.__init__ at 0x1103c69d0>, 'chirp': <function Chicken.chirp at 0x1103c6a60>, '__doc__': None}
=====>Bird
{'__module__': 'fluentpy.c8.simple_bird', 'feather': True, 'chirp': <function Bird.chirp at 0x1103c6940>, '__dict__': <attribute '__dict__' of 'Bird' objects>, '__weakref__': <attribute '__weakref__' of 'Bird' objects>, '__doc__': None}
{'__module__': 'fluentpy.c8.simple_bird', 'feather': True, 'chirp': <function Bird.chirp at 0x1103c6940>, '__dict__': <attribute '__dict__' of 'Bird' objects>, '__weakref__': <attribute '__weakref__' of 'Bird' objects>, '__doc__': None}


In [4]:
print(summer.age,summer.fly)

2 False


In [7]:
summer.fly=True
display(summer.fly)  # 对象的属性赋值时,只会查找本对象的__dict__,若无此属性,自动添加.
display(summer.__dict__)
display(Chicken.__dict__)

True

{'age': 2, 'fly': True}

mappingproxy({'__module__': 'fluentpy.c8.simple_bird',
              'fly': False,
              '__init__': <function fluentpy.c8.simple_bird.Chicken.__init__(self, age) -> None>,
              'chirp': <function fluentpy.c8.simple_bird.Chicken.chirp()>,
              '__doc__': None})

类的方法和函数的区别

In [44]:
from fluentpy.c8.simple_bird import *
from inspect import signature

def outer_test(a):
    print('out of class')

ck = Chicken(2)
print(dir(ck.chirp))

['__call__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__func__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__self__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [47]:
display(type(ck.chirp)) # chirp是method,又分为bound和unboud,属性方法;会隐含自动传入self
print(ck.chirp.__code__.co_varnames)
# TODO:signature(ck.chirp)会出错
for name,value in signature(ck.chirp.__call__).parameters.items():
    print(name,value)
ck.outer_test = outer_test
display(type(ck.outer_test)) #不会自动传入self
print(ck.outer_test.__code__.co_varnames)
print(signature(ck.outer_test))

method

()
args *args
kwargs **kwargs


function

('a',)
(a)
args *args
kwargs **kwargs


## super()的用法/解构--关联描述器
super在不同参数下可以代表unbound、bound到对象、bound到类型，这点在docstring当中其实写的比较清楚了，
Python3省略所有参数时候相当于super(self, class)，是bound到对象。而super(class, class)自然是绑定到了父类。
**实质上是对描述器的理解加深,一个描述器,通过__get__()的访问,可以将描述器放置到某实例,即绑定.**

In [ ]:
from fluentpy.c8.simple_bird import *

ck = Chicken(2)
display(Chicken.__mro__)
display(Chicken.__dict__)
display(Bird.__dict__)

# o = super()
# print(o)
# print(o.chirp)
o = super(Chicken, ck)  # ==类内部的super()的用法,mro链的内容由ck确定并绑定在ck上的Bird(Chicken的下一跳)的chirp方法.
print(o)
print(isinstance(o,super)) 
chirp1 = getattr(o,'chirp')
display(chirp1) # 已绑定至ck实例
print(o.chirp)
print(o.chirp())
print(o.funcname)
o = super(Chicken, Chicken)  # 同上注释,bound super object; requires issubclass(type2, type)
print(o)
chirp3=getattr(o,'chirp') # 未绑定至实例,为function
chirp4=o.chirp.__get__(None,Chicken) # 未绑定至实例,返回function本身
display(chirp3 is chirp4) #同一id
chirp2 = getattr(o,'chirp').__get__(ck) # function也是描述器,调用省略了self,绑定至ck,为bound method
display(chirp2) 
display(chirp1==chirp2)
display(chirp1 is chirp2)
print(o.chirp)
print(o.chirp(ck))
print(o.funcname())  # funcname()绑定在Chicken上,这里默认传入的cls 是 Chicken
print("=" * 5, ">no second param.")
o = super(Chicken)  # unbound super object
print(hasattr(o, "chirp"))
b1 = o.__get__(ck,type(ck)) #o.__get__(ck,type(ck)),可以把o当做Chicken中的描述器,使用ck访问,bound super object
b2 = o.__get__(ck)
b3 = o.__get__(None,type(ck))
print(b1)
print(b2)
print(b1==b2)
print(b3) # 返回装饰器o本身,b3 is o
print(o is b3)
print(b2.__thisclass__) # Chicken
print(b2.__self__) #绑定至ck为ck这个实例,绑定至Chicken为Chicken这个类
print(b2.__self_class__)
print(isinstance(b1,super))
print(b1.chirp) # dispatch to the method of Bird.chirp
# print(type(o).__dict__) #在类上访问__dict__才能看到super(Chicken)里的内容.
print(b1.feather) # 如果访问fly出错,因为Bird没有fly的属性.
b1 = o.__get__(Chicken) 
b2 = o.__get__(Chicken,type(Chicken))
print(b1)
print(b2)
print(b1 == b2)
print(b1.chirp)
print(b1.chirp.__get__(ck)) # 将function chirp绑定至实例ck
print(b2.__thisclass__) # Chicken
print(b2.__self__) #绑定至ck为ck这个实例,绑定至Chicken为Chicken这个类
print(b2.__self_class__)
o = super(Bird)
print(hasattr(o, "chirp"))

## Vector2d_v0: fulentpy C9.1
实现默认方法,不是每个类都需要这么实现,如果要借助内置方法如abs(),repr(),str()即可实现其方法,__iter__()使对象内容可迭代,即可用在类似于tuple()和list()之中.

In [6]:
import operator as op
from fluentpy.c9.vector2d_v0 import Vector2d

v = Vector2d(3,4)
display(abs(v))
display(op.eq(v,(3,4)))
display("{}({!r},{!r})".format(*[1,2]))

5.0

True

IndexError: Replacement index 2 out of range for positional args tuple

In [9]:
display(list(v))
len(list(v))

[3, 4]

2

In [50]:
v = Vector2d(3,4)
# 调用vector2d 自定义的__bytes__()
b = bytes(v)
display(b)

display(chr(b[0]))
display(b[1:])
v1 = Vector2d.frombytes(b)
display(v1)
display(v == v1)

***** in __bytes__()


b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

'a'

'd'

b'\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

Vector2d(3.0,4.0)

True

## 类属性的使用

## Py的特性:@property

### @property的基本使用
对属性first_name的访问均会通过getter和setter进行.

In [2]:
from pycookbook.c8.person import *

p = Person('coolfish')
display(p.first_name)
p.first_name='coolfish007'
display(p.first_name)

=====>设置实例属性
=====>读取实例属性


'coolfish'

=====>设置实例属性
=====>读取实例属性


'coolfish007'

直接给内部属性赋值,绕过setter

In [7]:
p._first_name = 42
display(p.first_name)

42

In [7]:
from pycookbook.c8.person import *

p = Person(42) #TypeError: Expected a string

### 特性是类属性
(*)特性被类直接访问赋值后,就地销毁.

In [3]:
from pycookbook.c8.person import *

print(vars(Person))
print(dir(Person.first_name))
print(Person.first_name)
#Person.first_name='coolfish008' #赋值类属性
p = Person('coolfish') #赋值实例属性
print(Person.first_name)
print(vars(p))
print('Person.first_name.fget(p):')
print(Person.first_name.fget(p)) # 重复运行时,Person.first_name已变为str
print('p.first_name:',p.first_name)
Person.first_name='class name' # 覆盖特性,特性first_name被销毁,不再通过getter取值,setter赋值
print(Person.first_name)
print(vars(p))
print(p.first_name)
print(p._first_name)

{'__module__': 'pycookbook.c8.person', '__init__': <function Person.__init__ at 0x103865940>, 'first_name': 'class name', '__dict__': <attribute '__dict__' of 'Person' objects>, '__weakref__': <attribute '__weakref__' of 'Person' objects>, '__doc__': None}
['__add__', '__class__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mod__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmod__', '__rmul__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'capitalize', 'casefold', 'center', 'count', 'encode', 'endswith', 'expandtabs', 'find', 'format', 'format_map', 'index', 'isalnum', 'isalpha', 'isascii', 'isdecimal', 'isdigit', 'isidentifier', 'islower', 'isnumeric', 'isprintable', 'isspace', 'istitle', 'isupper', 'join', 'ljust', 'lower', 'lstrip', 'maketr

AttributeError: 'str' object has no attribute 'fget'

### 使用特性延迟计算并缓存结果

In [5]:
from pycookbook.c8.c810lazy2 import *

c1 = Circle(2)

print(c1.area)

===== >lazy(self) 1>enter 
===== >lazy(self) 3>no attr  <pycookbook.c8.c810lazy2.Circle object at 0x111693460> _lazy_area <function Circle.area at 0x10fa65940>
===== >lazy(self) 4>func  <function Circle.area at 0x10fa65940> ('self',) 1
===== >area(self) 1>computing area
12.566370614359172


### (*)lazy2 @property 解构

In [2]:
from pycookbook.c8.c810lazy2 import *

c2 = Circle(3)
# 注意这个地方,需写成 类.method的方式,这样是一个function
# 这样调用时需显式的写成 method(self),手动传入self
# 否则若写成对象.method的方式如c2.perimeter,在lazy()中调用func(self),会自动传入self
# 这样,位置参数的数量就不一致了.
wrapped = Circle.perimeter
f = lazyproperty(wrapped)
print(dir(f))
print(f)
print(f.fget)
print(dir(f.fget))
print(f.fget(c2)) # property 绑定至c2

['__class__', '__delattr__', '__delete__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__isabstractmethod__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__set__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'deleter', 'fdel', 'fget', 'fset', 'getter', 'setter']
<function lazyproperty.<locals>.lazy at 0x10348cf70>
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
===== >lazy(self) 1>enter 
===== >lazy(self) 3>no at

特性是类属性,将特性f设置为Circle的类属性.  
之后可以直接在对象上访问属性perimeter.  
至此,解构彻底完成.

In [3]:
setattr(Circle,'perimeter',f)  # fltpy P509,除了直接给__dict__的赋值,这些方法都会调用处理特性的特殊方法.
c3=Circle(4)
print(c3.perimeter)
print('cached perimeter')
print(c3.perimeter)

===== >lazy(self) 1>enter 
===== >lazy(self) 3>no attr  <pycookbook.c8.c810lazy2.Circle object at 0x10348ba90> _lazy_perimeter <function Circle.perimeter at 0x1035d8a60>
===== >lazy(self) 4>func  <function Circle.perimeter at 0x1035d8a60> ('self',) 1
===== >perimeter() 1>computing perimeter
25.132741228718345
cached perimeter
===== >lazy(self) 1>enter 
===== >lazy(self) 2>has attr  <pycookbook.c8.c810lazy2.Circle object at 0x10348ba90> _lazy_perimeter
25.132741228718345


In [2]:
from pycookbook.c8.c810lazy2 import *

c2 = Circle(3)
c2.perimeter

===== >lazy(self) 1>enter 
===== >lazy(self) 3>no attr  <pycookbook.c8.c810lazy2.Circle object at 0x10937a3a0> _lazy_perimeter <function Circle.perimeter at 0x109c66a60>
===== >lazy(self) 4>func  <function Circle.perimeter at 0x109c66a60> ('self',) 1
===== >perimeter() 1>computing perimeter


18.84955592153876

## Py的描述器
类的实例类型;类的type()是type类型.

In [7]:
from pycookbook.c8.c813checkattri import *

name = SizedString('name',size=8)
display(type(name))
display(isinstance(name,Descriptor))
display(isinstance(name,Typed))
display(isinstance(name,String))
display(isinstance(name,MaxSized))

pycookbook.c8.c813checkattri.SizedString

True

True

True

True

In [12]:
from pycookbook.c8.c813checkattri import *

ui = UnsignedInteger() # () 代表已构造类的实例
display(type(ui))
display(isinstance(ui,Descriptor))

ui = UnsignedInteger
display(type(ui))
display(isinstance(ui,Descriptor))
# 使用参数名称,用ui构造一个UnsignedInteger对象实例
display(isinstance(ui('shares'),Descriptor))
# 显示UnsignedInteger 的__set__来自于哪里
display(ui.__mro__)
display(ui.__set__)
# 显示__dict__
ui('share').__dict__

pycookbook.c8.c813checkattri.UnsignedInteger

True

type

False

True

(pycookbook.c8.c813checkattri.UnsignedInteger,
 pycookbook.c8.c813checkattri.Integer,
 pycookbook.c8.c813checkattri.Typed,
 pycookbook.c8.c813checkattri.Unsigned,
 pycookbook.c8.c813checkattri.Descriptor,
 object)

<function pycookbook.c8.c813checkattri.Typed.__set__(self, ins, value)>

{'name': 'share'}

### 多重继承

MRO 方法执行顺序

In [2]:
import logging
from pycookbook.c8.c813checkattri import *

logging.basicConfig(level=logging.DEBUG)  # 之后再设置level,需要重启jupyter
logging.info('hi')
display(SizedString.__mro__)
name = SizedString('name',size=8)
print(dir(name))
name.__set__(name,6)  #按照__mro__的顺序,同一方法,进入typed,进入maxsized,进入descriptor执行,执行maxsized剩余的,执行typed剩余的.

INFO:root:hi


(pycookbook.c8.c813checkattri.SizedString,
 pycookbook.c8.c813checkattri.String,
 pycookbook.c8.c813checkattri.Typed,
 pycookbook.c8.c813checkattri.MaxSized,
 pycookbook.c8.c813checkattri.Descriptor,
 object)

DEBUG:root:=>init in MaxSized.
DEBUG:root:=>init in Descriptor.
DEBUG:root:=>set in Typed.
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__set__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'expected_type', 'name', 'size']


TypeError: can only concatenate str (not "type") to str

### 描述器的使用
描述器做为类属性

In [7]:
import logging
from pycookbook.c8.c813checkattri import *

logging.basicConfig(level=logging.WARN)

class Stock1:
    name = SizedString('name',size=8)
    shares = UnsignedInteger('shares')
    price = UnsignedFloat('price')


    def __init__(self,name,shares,price):
        self.name = name #描述器和实例属性同名,描述器覆盖实例属性.
        self.shares = shares
        self.price = price

    def __getattr__(self,name):
        print(self,name)
        return 'default'

display(type(Stock1)) #type
display(isinstance(Stock1,type))

DEBUG:root:=>init in MaxSized.
DEBUG:root:=>init in Descriptor.
DEBUG:root:=>init in Descriptor.
DEBUG:root:=>init in Descriptor.


type

True

关联类的属性和属性覆盖(调用顺序)

In [ ]:
stock = Stock1('hello',5,3.0)
# display(Stock1.name.__get__(None,Stock1)) #描述器无__get__()方法,直接访问__dict__的内容.
display(stock.haha) # 无haha属性,调用__getattr__()
stock.name='hello2'
display(stock.__dict__['name'],stock.name,Stock1.name)
Stock1.name='haha3'  # 销毁了描述器,变成普通的str.
display(stock.name,Stock1.name)

In [8]:
stock = Stock1('hello',5,3.0)
stock.name = 'hello1'
display(stock.name)
stock.price='hellohello' #error:size must be < 8

DEBUG:root:=>set in Typed.
DEBUG:root:=>set in MaxSized.
DEBUG:root:=>set in Descriptor.
DEBUG:root:=>set in Descriptor end.
DEBUG:root:=>set in MaxSized end.
DEBUG:root:=>set in Typed end.
DEBUG:root:=>set in Typed.
DEBUG:root:=>set in Descriptor.
DEBUG:root:=>set in Descriptor end.
DEBUG:root:=>set in Typed end.
DEBUG:root:=>set in Typed.
DEBUG:root:=>set in Descriptor.
DEBUG:root:=>set in Descriptor end.
DEBUG:root:=>set in Typed end.
DEBUG:root:=>set in Typed.
DEBUG:root:=>set in MaxSized.
DEBUG:root:=>set in Descriptor.
DEBUG:root:=>set in Descriptor end.
DEBUG:root:=>set in MaxSized end.
DEBUG:root:=>set in Typed end.


'hello1'

DEBUG:root:=>set in Typed.


TypeError: expected <class 'float'>

### 描述器实现及使用:混入+装饰器的方式
使用装饰器check_attri+混入的方式做为类变量的检查器

In [14]:
import logging
from pycookbook.c8.c813checkattri import *

logging.basicConfig(level=logging.INFO)

@check_attri(name=SizedString(size=8),shares=UnsignedInteger,price=UnsignedFloat)
class Stock2:
    def __init__(self,name,shares,price):
        self.name = name
        self.shares = shares
        self.price = price

s2 = Stock2('hello',5,3.0)
display(s2.name)

INFO:root:k=name,v=<pycookbook.c8.c813checkattri.SizedString object at 0x10d3570a0>
INFO:root:True
INFO:root:instance of Descriptor,v is <class 'pycookbook.c8.c813checkattri.SizedString'>
INFO:root:k=shares,v=<class 'pycookbook.c8.c813checkattri.UnsignedInteger'>
INFO:root:False
INFO:root:not instance of Descriptor,v is <class 'type'>
INFO:root:k=price,v=<class 'pycookbook.c8.c813checkattri.UnsignedFloat'>
INFO:root:False
INFO:root:not instance of Descriptor,v is <class 'type'>


'hello'

### 描述器的实现及使用:纯装饰器的方式

In [2]:
from pycookbook.c8.c813checkattri_decro import *

f = typed(int)
display(f)

functools.partial(<function typed at 0x10deaeca0>, <class 'int'>)

In [3]:
from pycookbook.c8.c813checkattri_decro import *

f = Integer('age')
display(f)

In [2]:
from pycookbook.c8.c813checkattri_decro import *

print('expected: '+str(Integer))
if not 0:
    raise TypeError(f'expected: {repr(float)}')

expected: <class 'pycookbook.c8.c813checkattri_decro.Integer'>


TypeError: expected: <class 'float'>

In [6]:
from pycookbook.c8.c813checkattri_decro import *

logging.basicConfig(level=logging.INFO)
class Stock3:
    age1 = UnsignedI1('age1')
    age2 = UnsignedI2('age2')

    def __init__(self,age1,age2):
        self.age1 = age1
        self.age2 = age2 

s3 = Stock3('hi',3.0)
display(s3.age1,s3.age2)

TypeError: '<' not supported between instances of 'str' and 'int'

## 单例模式 pycb C8.25
### 依然可以调用__init__()

In [ ]:
from pycookbook.c8.c825singleton import Spam

s = Spam('hello')
s1 = Spam('hello')
display(s is s1)
display(list(Spam._spam_cache))
del s 
display(list(Spam._spam_cache))
del s1
display(list(Spam._spam_cache)) # 引用都售出后,在WeakValueDictionary中移除.

### 隐藏__init__()

In [ ]:
from pycookbook.c8.c825sing1 import Spam2

s = Spam2.get_instance('hello')
s1 = Spam2.get_instance('hello')
display(s is s1)

s = Spam2('hello') # __init__()出错.

### 使用元类创建单例或单例的缓存 pycb C9.13
按关键字参数ids创建单例的缓存.

In [ ]:
from pycookbook.c8.c825sing2c913 import Spam3
s = Spam3('hello',42,ids=123)
s1 = Spam3('hello',42,ids=123)
s2 = Spam3('hello1',42,ids=123)
s3 = Spam3('hello',43,ids=456)
display(s is s1)
display(s1 is s2)
display(s is s3)

# 6.Py3 字符串与编码

## chr/bytearray/

In [80]:
# 默认编码是unicode_escape
display(chr(97))
display(chr(0x61)) #0x61==97
display(ord('你'))
display(chr(20320))
display(chr(0x4f60))# 两个8位

'a'

'a'

20320

'你'

'你'

In [60]:
display('abc'.encode('ascii'))
display('abc'.encode('utf-8'))
display('你好'.encode('utf-8'))
display('你好'.encode('unicode_escape'))

b'abc'

b'abc'

b'\xe4\xbd\xa0\xe5\xa5\xbd'

b'\\u4f60\\u597d'

In [59]:
display(b'abc'.decode('ascii'))

'abc'

In [45]:
from array import array

# typecode 不同,bytes()也不同
# b:1个byte / f:4个 / d:8个 / u:2个/
a = array('l',[3,4])
display(a)
b = bytes(a)
display(b)
x=3
x.to_bytes(8,'little')

array('l', [3, 4])

b'\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00'

b'\x03\x00\x00\x00\x00\x00\x00\x00'

In [47]:
b = bytes('hello world',encoding='iso8859-1')
display(b)
# 可见字节
b = bytes('hello world',encoding='ascii')
display(b,len(b))
b = bytes('你好',encoding='gbk')
display(b,len(b))
b = bytes('你好',encoding='utf-8')
display(b)

b'hello world'

b'hello world'

11

b'\xc4\xe3\xba\xc3'

4

b'\xe4\xbd\xa0\xe5\xa5\xbd'

# 7. 协程

## 7.1 yield与yield from

In [6]:
def func1():
    print('in func1()')
    ret = yield request("http://test.com/foo")
    ret = yield func2(ret)
    return ret

def func2(data):
    print('in func2()')
    result = yield request("http://test.com/"+data)
    return result

def request(url): 
    """ 这里模拟返回一个io操作，包含io操作的所有信息，这里用字符串简化代替.
    request 如果注册给ioloop,即io完成后进行唤醒. """
    print(f'in request().before request yield:iojob of {url}')
    result = yield "return url:iojob of %s" % url
    print('after retuqest yield')
    return result

In [7]:
g = request('bar')
ret = g.send(None)
print('*'*5,ret)
ret = g.send('end')
print(ret)

in request().before request yield:iojob of bar
***** return url:iojob of bar
after retuqest yield


StopIteration: end

### fltpy 16.18示例
子生成器func1运行,func1内部嵌套第二层生成器没有运行.

In [35]:
from inspect import isgenerator

def wrapper(gen):
    _i = iter(gen) #func1
    try:
        _y = next(_i)
    except StopIteration as _e:
        _r = _e.value
    else:
        while True:
            _s = yield _y
            try:
                _y = _i.send(_s)
            except StopIteration as _e:
                _r = _e.value
                break
    print('--',_r)
    return _r

w = wrapper(func1())
params = [None,'bar','end']
result = None
for p in params:
    # result = w.send(p)
    print(w.send(p)) #w也是生成器,func1的stopiteration捕捉了,w也执行完了,没有捕捉;
    # if isgenerator(result):
    #     w = result

in func1()
<generator object request at 0x1108bdf20>
<generator object func2 at 0x10f7892e0>
stopiteration
-- end


StopIteration: end

### 模拟yield from的执行(来自于pocket收藏:python协程的本质
子生成器嵌套的触发不断运行

In [30]:
from inspect import isgenerator
from inspect import getgeneratorstate

def wrapper(gen):
    print('in wrapper gen:',gen)
    stack = []
    stack.append(gen)
    result = None
    while True:
        item = stack[-1]
        if(isgenerator(item)):
            try:
                print(item,getgeneratorstate(item))
                child = item.send(result) # 在func1第一个yield处暂停,返回生成器request(url);
                stack.append(child)
                result = None
                continue
            except StopIteration as e:
                print('StopIteration',e.value) #直至request执行结束,捕获异常;
                result = e.value
        else:
            print('return request yield value.')
            result = yield item # 返回request(url)中yield右侧的值,并等待调用方发送参数
        stack.pop() # 当返回非生成器或执行完后,弹出栈顶的元素;
        if not stack:
            print('finish.')
            return result

w = wrapper(func1())
print('*'*5,'client start...',isgenerator(w)) # 如果w也是generator,先执行此行;w在send(None)预激时执行;
try:
    w.send(None)
    print('*'*5,'execute in client...')
    w.send('bar')
    print('*'*5,'execute in client...')
    w.send('the end')
except StopIteration as e:
    print('The final result:'+e.value)

***** client start... True
in wrapper gen: <generator object func1 at 0x1108c0eb0>
<generator object func1 at 0x1108c0eb0> GEN_CREATED
in func1()
<generator object request at 0x1108c0f90> GEN_CREATED
in request().before request yield:iojob of http://test.com/foo
return request yield value.
***** execute in client...
<generator object request at 0x1108c0f90> GEN_SUSPENDED
after retuqest yield
StopIteration bar
<generator object func1 at 0x1108c0eb0> GEN_SUSPENDED
<generator object func2 at 0x1108c0f90> GEN_CREATED
in func2()
<generator object request at 0x1108c0b30> GEN_CREATED
in request().before request yield:iojob of http://test.com/bar
return request yield value.
***** execute in client...
<generator object request at 0x1108c0b30> GEN_SUSPENDED
after retuqest yield
StopIteration the end
<generator object func2 at 0x1108c0f90> GEN_SUSPENDED
StopIteration the end
<generator object func1 at 0x1108c0eb0> GEN_SUSPENDED
StopIteration the end
finish.
The final result:the end


## 使用yield from代替yield

In [6]:
from types import coroutine

def func1():
    print('in func1()')
    ret = yield from request("http://test.com/foo") # 如果去掉from,在这里暂停
    ret = yield from func2(ret)
    return ret

def func2(data):
    print('in func2()')
    result = yield from request("http://test.com/"+data)
    return result

async def func3(data):
    print('in func3()')
    await request_co("http://test.com/"+data)
    return 'end func3'

def request(url):
    # 这里模拟返回一个io操作，包含io操作的所有信息，这里用字符串简化代替
    print(f'before request yield:iojob of {url}')
    result = yield "return url:iojob of %s" % url
    print('after retuqest yield:',result)
    return result

@coroutine
def request_co(url):
    # 这里模拟返回一个io操作，包含io操作的所有信息，这里用字符串简化代替
    print(f'before request yield:iojob of {url}')
    yield "return url:iojob of %s" % url
    print('after retuqest yield:')



In [21]:
g = func2('bar')
ret = g.send(None)
print('*'*5,ret)
ret = g.send('end')
print(ret)

in func2()
before request yield:iojob of http://test.com/bar
***** return url:iojob of http://test.com/bar
after retuqest yield: end


StopIteration: end

### 调度器模拟,asyncio-async/await 执行

In [17]:
import traceback

async def func4():
    """ 管控func3()的执行,执行2次.执行完之后抛出StopIteration. """
    i = 2
    while True:
        if(i==0):
            break
        ret = await func3('bar+'+str(i))
        print('*'*3*i,ret)
        i-=1

co = func4()
print(co)
# 下边相当与一个调度器
while True:
    try:
        print('-----send co signal.') #执行3次;第二次执行推动yield向下并
        co.send(None) #推动协程运行
    except StopIteration as e:
        traceback.print_exc()
        break

<coroutine object func4 at 0x1094c47c0>
in func3()
before request yield:iojob of http://test.com/bar+2
***after co send(None)
after retuqest yield:
end func3()
****** end func3
in func3()
before request yield:iojob of http://test.com/bar+1
***after co send(None)
after retuqest yield:
end func3()
*** end func3



### yield from的执行

In [149]:
from inspect import getgeneratorstate
from inspect import isgenerator

g = func1()
print('func1:',isgenerator(g),getgeneratorstate(g))
result = g.send(None)
print('*'*5,result,',sending param')
result = g.send('bar')
print('*'*5,result,',sending param')
g.send('the end')

func1: True GEN_CREATED
in func1()
before request yield:iojob of http://test.com/foo
***** return url:iojob of http://test.com/foo ,sending param
after retuqest yield: bar
in func2()
before request yield:iojob of http://test.com/bar
***** return url:iojob of http://test.com/bar ,sending param
after retuqest yield: the end


StopIteration: the end

In [46]:
from inspect import getgeneratorstate
from inspect import isgenerator

def wrapper(gen):
    yield from gen
w = wrapper(func1())
ret = w.send(None)
print('*'*5,'execute in client...',ret)
ret = w.send('bar')
print('*'*5,'execute in client...',ret)

in func1()
before request yield:iojob of http://test.com/foo
***** execute in client... return url:iojob of http://test.com/foo
after retuqest yield: bar
in func2()
before request yield:iojob of http://test.com/bar
***** execute in client... return url:iojob of http://test.com/bar
